In [1]:
# 환경설정

!pip3 install transformers
!pip3 install torch
!pip3 install konlpy

     |████████████████████████████████| 3.1 MB 12.8 MB/s 
     |████████████████████████████████| 3.3 MB 35.6 MB/s 
     |████████████████████████████████| 59 kB 6.4 MB/s 
     |████████████████████████████████| 596 kB 39.5 MB/s 
     |████████████████████████████████| 895 kB 37.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 19.4 MB 1.4 MB/s 
     |████████████████████████████████| 448 kB 49.1 MB/s 
     |████████████████████████████████| 86 kB 5.7 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [2]:
# 시험데이터셋 수집

!wget https://raw.githubusercontent.com/aifactory-team/hanryubank/main/dataset_test.csv

--2021-11-19 05:29:57--  https://raw.githubusercontent.com/aifactory-team/hanryubank/main/dataset_test.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 21935 (21K) [text/plain]
Saving to: ‘dataset_test.csv’

dataset_test.csv    100%[===================>]  21.42K  --.-KB/s    in 0s      

2021-11-19 05:29:57 (99.8 MB/s) - ‘dataset_test.csv’ saved [21935/21935]



In [3]:
# 모듈 설정

from transformers import ElectraTokenizer, ElectraForQuestionAnswering, pipeline
import pandas as pd

tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-small-v2-distilled-korquad-384")
model = ElectraForQuestionAnswering.from_pretrained("monologg/koelectra-small-v2-distilled-korquad-384")
qa = pipeline("question-answering", tokenizer=tokenizer, model=model)

import konlpy
okt = konlpy.tag.Okt()

Downloading:   0%|          | 0.00/249k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/472 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/52.3M [00:00<?, ?B/s]

In [4]:
# 입력된 질문(query)에 해당하는 유사한 질문을 찾고, 그 유사한 질문에 해당하는 지문을 검색하는 함수

def get_context_and_question(query):

    context = ''
    question = ''

    return context, question

In [5]:
# 지문과 질문을 입력하여 정답을 얻는 함수

def get_answer(context, question):
    result = qa({"question": question, "context": context})
    score = result["score"]
    start = result["start"]
    end = result["end"]
    answer = result["answer"]
    return answer, start, end, score

In [6]:
# 해답의 어미를 바꾸는 함수

def modify_answer(before_answer):

    document = before_answer
    
    clean_words = []

    for word in okt.pos(document, stem=True):
        if word[1] not in ['Josa', 'Eomi', 'Punctuation']:
            clean_words.append(word[0])
    
    clean_words.append('이라네.')
    after_answer = ''.join(clean_words)

    return after_answer

In [ ]:
if __name__ == "__main__":
    df = pd.read_csv('dataset_test.csv')
    
    for idx, row in df.iterrows():
        it_context = row['Context (지문)']
        it_question = row['Q1 (질문1)']
        it_ans = row['Ans (답변)']

        answer, start, end, score = get_answer(it_context, it_question)

        print("[" + str(idx) + "]")
        print("question: " + it_question)
        print("real: " + it_ans)
        print("predict: {0} ({1},{2},{3})".format(answer, start, end, score))
        print("service: " + modify_answer(answer))

[0]
question: 세종대왕이 무엇을 만들었는가
real: 훈민정음
predict: 훈민정음을 (20,25,0.9984866976737976)
service: 훈민정음이라네.
[1]
question: 조세에 관하여 무엇을 시행했는가
real: 여론 조사
predict: 여론 조사를 (30,36,0.9997721314430237)
service: 여론조사이라네.
[2]
question: 훈민정음을 창제한 이유
real: 애민 정신
predict: 애민 정신을 (320,326,0.6614944338798523)
service: 애민정신이라네.
[3]
question: 농부를 위해 만든 것은
real: 농사직설
predict: 농사직설을 (124,129,0.9998931884765625)
service: 농사직설이라네.
[4]
question: 어렸을 때부터 똑똑했던 이유는
real: 책읽기
predict: 책읽기를 (48,52,0.9598397016525269)
service: 책읽다이라네.
[5]
question: 여가 생활에 어떤 것을 했을까?
real: 음악
predict: 음악을 (132,135,0.9993412494659424)
service: 음악이라네.
[6]
question: 한글의 다른 이름은 무엇인가?
real: 언문
predict: 언문이라 (16,20,0.9977233409881592)
service: 언문이라네.
[7]
question: 언제 공식적으로 한글이 되었나
real: 1894년
predict: 1894년 (0,5,0.9279113411903381)
service: 1894년이라네.
[8]
question: 한글이라고 이름을 바꾼 사람은
real: 주시경
predict: 주시경이 (310,314,0.9978508353233337)
service: 주시경이라네.
[9]
question: 어떤 방식으로 정치를 하였나
real: 토론
predict: 토론이었다. (32,38,0.9935980439186096)
service: 토론이